In [49]:
import numpy as np

def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    # Normalize both axes
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    # Generate all permutations with sign changes of axis1 to account for symmetry
    axis_permutations = [
        axis1,
        [-axis1[0], axis1[1], axis1[2]],
        [axis1[0], -axis1[1], axis1[2]],
        [axis1[0], axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], axis1[2]],
        [-axis1[0], axis1[1], -axis1[2]],
        [axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
    ]
    
    # Check if axis2 is equivalent to any of the permutations of axis1 within the specified angle threshold
    for perm in axis_permutations:
        # Calculate the cosine of the angle between the two vectors
        cos_angle = np.dot(perm, axis2)
        
        # Ensure the value is within the valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate the angle in degrees
        angle = np.degrees(np.arccos(cos_angle))

        
        # Check if the angle is less than the threshold
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            print(f"Angle between Axis and CSL Axis {perm:}: {angle:.2f}°")
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=2.0):
    """
    Determine if the given rotation axis and misorientation angle correspond to a Σ boundary,
    considering the Brandon criterion for deviation and axis symmetry.
    
    Parameters:
    axis (list or tuple): Rotation axis (e.g., [1, 1, 1])
    misorientation_angle (float): Misorientation angle (in degrees)
    structure_type (str): Crystal structure type ('FCC' or 'BCC')
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether it is a Σ boundary
    int: Corresponding Σ value (if it is a Σ boundary)
    float: Actual deviation (in degrees)
    """
    # Predefined Σ values with corresponding rotation axes and misorientation angles for both FCC and BCC
    sigma_data = {
        3: {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
            'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        5: {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
            'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        7: {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
            'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        9: {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
            'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        11: {'FCC': [{'axis': [1, 1, 1], 'angle': 50.48}],
             'BCC': [{'axis': [1, 1, 1], 'angle': 50.48}]},
        13: {'FCC': [{'axis': [1, 0, 0], 'angle': 27.8}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 27.8}]},
        17: {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        19: {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
             'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        21: {'FCC': [{'axis': [1, 0, 1], 'angle': 21.79}],
             'BCC': [{'axis': [1, 0, 1], 'angle': 21.79}]},
        23: {'FCC': [{'axis': [1, 1, 1], 'angle': 20.1}],
             'BCC': [{'axis': [1, 1, 1], 'angle': 20.1}]},
        27: {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
             'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        29: {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
    }
    
    # Normalize the given rotation axis
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            # Check if the given axis is equivalent to the predefined axis
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                
                # Calculate the deviation
                deviation = abs(misorientation_angle - predefined_angle)
                
                # Calculate the Brandon criterion threshold
                brandon_threshold = 15 / np.sqrt(sigma)
                
                # Check if the misorientation angle matches within the Brandon criterion threshold
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Example usage
axis1 = [17, 17, -16]
misorientation_angle1 = 59.6
structure_type = 'FCC'  # 'FCC' or 'BCC'

is_sigma1, sigma_value1, deviation1 = is_sigma_boundary(axis1, misorientation_angle1, structure_type)
if is_sigma1:
    print(f"This is a Σ{sigma_value1} boundary with a deviation of {deviation1:.2f} degrees")
else:
    print("This is not a Σ boundary")

axis2 = [16, 16, -16]
misorientation_angle2 = 60

is_sigma2, sigma_value2, deviation2 = is_sigma_boundary(axis2, misorientation_angle2, structure_type)
if is_sigma2:
    print(f"This is a Σ{sigma_value2} boundary with a deviation of {deviation2:.2f} degrees")
else:
    print("This is not a Σ boundary")


Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 1.62°
This is a Σ3 boundary with a deviation of 0.40 degrees
Angle between Axis and CSL Axis [0.5773502691896258, 0.5773502691896258, -0.5773502691896258]: 0.00°
This is a Σ3 boundary with a deviation of 0.00 degrees


In [51]:
To  do
euler_angles1 = (112, 22.5, 74.5)
euler_angles2 = (129, 154.3, 250.9)

Orientation 1 Euler angles: (112.0, 22.5, 74.5)
Orientation 2 Euler angles: (129.0, 154.3, 250.9)
Misorientation angle: 59.6
Rotation axis: [16, -17, 17]
sogma 3  deviation 1.1

import numpy as np
from scipy.spatial.transform import Rotation as R

def rotation_matrix_from_euler(euler_angles):
    """
    Generate a rotation matrix from Euler angles.
    
    Parameters:
    euler_angles (tuple): Euler angles in degrees (phi1, Phi, phi2)
    
    Returns:
    numpy array: The rotation matrix
    """
    return R.from_euler('zxz', euler_angles, degrees=True).as_matrix()

def calculate_rotation_axis_and_angle(orientation1, orientation2):
    """
    Calculate the rotation axis and misorientation angle between two orientations.
    
    Parameters:
    orientation1, orientation2 (numpy arrays): The orientations as rotation matrices
    
    Returns:
    numpy array: The rotation axis
    float: The misorientation angle in degrees
    """
    rotation_matrix = np.dot(orientation2.T, orientation1)
    rotation = R.from_matrix(rotation_matrix)
    rotation_axis = rotation.as_rotvec()
    rotation_angle = np.degrees(np.linalg.norm(rotation_axis))
    rotation_axis_normalized = rotation_axis / np.linalg.norm(rotation_axis)
    return rotation_axis_normalized, rotation_angle

def are_axes_equivalent(axis2, axis1, angle_threshold=2.0):
    """
    Check if two axes are equivalent considering crystal symmetry and slight deviations by comparing their misorientation angle.
    
    Parameters:
    axis1, axis2 (numpy arrays): The axes to compare
    angle_threshold (float): The angle threshold in degrees for considering the axes equivalent
    
    Returns:
    bool: Whether the axes are equivalent
    """
    axis1 = axis1 / np.linalg.norm(axis1)
    axis2 = axis2 / np.linalg.norm(axis2)
    
    axis_permutations = [
        axis1,
        [-axis1[0], axis1[1], axis1[2]],
        [axis1[0], -axis1[1], axis1[2]],
        [axis1[0], axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], axis1[2]],
        [-axis1[0], axis1[1], -axis1[2]],
        [axis1[0], -axis1[1], -axis1[2]],
        [-axis1[0], -axis1[1], -axis1[2]],
    ]
    
    for perm in axis_permutations:
        cos_angle = np.dot(perm, axis2)
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        angle = np.degrees(np.arccos(cos_angle))
        
        if angle <= angle_threshold or (180 - angle) <= angle_threshold:
            return True
    
    return False

def is_sigma_boundary(axis, misorientation_angle, structure_type='FCC', angle_threshold=2.0):
    sigma_data = {
        3: {'FCC': [{'axis': [1, 1, 1], 'angle': 60}],
            'BCC': [{'axis': [1, 1, 1], 'angle': 60}]},
        5: {'FCC': [{'axis': [1, 0, 0], 'angle': 36.87}],
            'BCC': [{'axis': [1, 0, 0], 'angle': 36.87}]},
        7: {'FCC': [{'axis': [1, 1, 1], 'angle': 38.21}],
            'BCC': [{'axis': [1, 1, 1], 'angle': 38.21}]},
        9: {'FCC': [{'axis': [1, 1, 0], 'angle': 38.94}],
            'BCC': [{'axis': [1, 1, 0], 'angle': 38.94}]},
        11: {'FCC': [{'axis': [1, 1, 1], 'angle': 50.48}],
             'BCC': [{'axis': [1, 1, 1], 'angle': 50.48}]},
        13: {'FCC': [{'axis': [1, 0, 0], 'angle': 27.8}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 27.8}]},
        17: {'FCC': [{'axis': [1, 0, 0], 'angle': 28.07}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 28.07}]},
        19: {'FCC': [{'axis': [1, 1, 0], 'angle': 26.53}],
             'BCC': [{'axis': [1, 1, 0], 'angle': 26.53}]},
        21: {'FCC': [{'axis': [1, 0, 1], 'angle': 21.79}],
             'BCC': [{'axis': [1, 0, 1], 'angle': 21.79}]},
        23: {'FCC': [{'axis': [1, 1, 1], 'angle': 20.1}],
             'BCC': [{'axis': [1, 1, 1], 'angle': 20.1}]},
        27: {'FCC': [{'axis': [1, 1, 0], 'angle': 31.59}],
             'BCC': [{'axis': [1, 1, 0], 'angle': 31.59}]},
        29: {'FCC': [{'axis': [1, 0, 0], 'angle': 43.6}],
             'BCC': [{'axis': [1, 0, 0], 'angle': 43.6}]},
    }
    
    axis = np.array(axis)
    
    for sigma, configs in sigma_data.items():
        if structure_type not in configs:
            continue
        for config in configs[structure_type]:
            predefined_axis = np.array(config['axis'])
            
            if are_axes_equivalent(axis, predefined_axis, angle_threshold):
                predefined_angle = config['angle']
                deviation = abs(misorientation_angle - predefined_angle)
                brandon_threshold = 15 / np.sqrt(sigma)
                
                if deviation <= brandon_threshold:
                    return True, sigma, deviation
    
    return False, None, None

# Example usage with two orientations given as Euler angles (phi1, Phi, phi2)
euler_angles1 = (112, 22.5, 74.5)
euler_angles2 = (129, 154.3, 250.9)

orientation1 = rotation_matrix_from_euler(euler_angles1)
orientation2 = rotation_matrix_from_euler(euler_angles2)

rotation_axis, misorientation_angle = calculate_rotation_axis_and_angle(orientation1, orientation2)

print(f"Rotation Axis: {rotation_axis}")
print(f"Misorientation Angle: {misorientation_angle:.2f}°")

is_sigma, sigma_value, deviation = is_sigma_boundary(rotation_axis, misorientation_angle)

if is_sigma:
    print(f"This is a Σ{sigma_value} boundary with a deviation of {deviation:.2f} degrees")
else:
    print("This is not a Σ boundary")


Rotation Axis: [-0.84634668  0.53181496 -0.0294982 ]
Misorientation Angle: 179.02°
This is not a Σ boundary
